In [4]:
import re
import pandas as pd

Load database

In [5]:
df = pd.read_csv('data/deertrail_raw.csv')
df.head()

,CASNumber,ParameterName,Value,UnitName,DetectionLimit,DilutionFactor,MDL,Method,SampleNumber,DataQualifier,CollectionTime,Laboratory,Location
0,11097-69-1,PCB-1254,-99.0,ug/L,0.48,1.0,0.20,SW-846 8082,DA29041-1,NaN,2020-09-22 10:30:00,SGS Wheatridge,Cell4-LCS
1,115-90-2,Fensulfothion,-99.0,ug/L,4.00,1.0,0.74,SW-846 8141A,DA29041-1,NaN,2020-09-22 10:30:00,SGS Wheatridge,Cell4-LCS
2,53494-70-5,Endrin ketone,-99.0,ug/L,0.20,10.0,0.05,SW-846 8081A,DA29041-1,NaN,2020-09-22 10:30:00,SGS Wheatridge,Cell4-LCS
3,959-98-8,Endosulfan I,-99.0,ug/L,0.20,10.0,0.10,SW-846 8081A,DA29041-1,NaN,2020-09-22 10:30:00,SGS Wheatridge,Cell4-LCS
4,33213-65-9,Endosulfan II,-99.0,ug/L,0.20,10.0,0.10,SW-846 8081A,DA29041-1,NaN,2020-09-22 10:30:00,SGS Wheatridge,Cell4-LCS


Format dataframe, sort by date

In [6]:
df.rename(columns={'CollectionTime': 'Time',
                   'ParameterName': 'Parameter', 'UnitName': 'Unit', 'CASNumber': 'CAS'}, inplace=True)
df['Time'] = pd.to_datetime(df['Time'])
df.sort_values('Time', inplace=True)
df.reset_index(inplace=True, drop=True)

Remove leading and trailing whitespace

In [7]:
df['CAS'] = df['CAS'].str.strip()

ND Value as '-99.0' to None

In [8]:
df.loc[df['Value'] < 0, 'Value'] = None
df.loc[(df['Value'].isna() & (df['DataQualifier'].isna())),
       ['DataQualifier']] = "ND"
df.loc[df['DetectionLimit'] < 0, 'DetectionLimit'] = None
df.loc[df['MDL'] < 0, 'MDL'] = None

Method as 'Unknown', 'UNK' or 'Unk'

In [9]:
df.loc[df['Method'].str.contains(r'Unk|Unknown', na=False, flags=re.IGNORECASE), 'Method'] = None

Laboratory as 'Unknown Laboratory'

In [10]:
df.loc[df['Laboratory'].str.contains(
    r'Unknown Laboratory|Unknown', na=False, flags=re.IGNORECASE), 'Laboratory'] = None

SampleNumber

In [11]:
df.loc[df['SampleNumber'].str.contains(
    r'Unknown', na=False, flags=re.IGNORECASE), 'SampleNumber'] = None

Title case for all caps?

In [12]:
df.loc[df['Parameter'].str.isupper(), 'Parameter'] = df['Parameter'].str.title()

ValueError: Cannot mask with non-boolean array containing NA / NaN values

', Total' to '(Total)'

In [ ]:
df.loc[(df['Parameter'].str.contains(r', Total', flags=re.IGNORECASE)), ['Parameter']] = df['Parameter'].str.replace(
    r', Total', r' (Total)', flags=re.IGNORECASE)

df.loc[(df['Parameter'].str.contains(r', Dissolved', flags=re.IGNORECASE)), ['Parameter']] = df['Parameter'].str.replace(
    r', Dissolved', r' (Dissolved)', flags=re.IGNORECASE)

Move field in parameter name to method if empty

In [ ]:
df.loc[(df['Parameter'].str.contains(r', Field', flags=re.IGNORECASE))
       & (df['Method'].isna()), ['Method']] = "Field"
df.loc[(df['Parameter'].str.contains(r', Field', flags=re.IGNORECASE)), ['Parameter']] = df['Parameter'].str.replace(
    r', Field', r'', flags=re.IGNORECASE)
df.head(25)